In [ ]:
from pathlib import Path
from operator import itemgetter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from rich.progress import track
from IPython.display import clear_output

import data_loader.data_loaders as module_data
import model.model as module_arch
from configs import configs
from utils.plot_utils import plot_signatures
from notebooks.helpers import import_artifacts_from_runID, load_ids_from_registry, get_plot_name
from utils.utils import ensure_dir

In [ ]:
def load_test_df(run_id):
	artifacts = import_artifacts_from_runID(run_id)
	data_loader, config = itemgetter("data_loader", "config")(artifacts)

	signatures_list = []
	targets_test = []
	labels_test = []
	imagings_test = []
	with torch.no_grad():
		for data, target, metadata in track(data_loader, description="Loading data..."):
			signature = torch.mean(data, dim=(2, 3))
			signatures_list.append(signature.detach().cpu().numpy().flatten())
			targets_test.append(target.numpy())
			labels_test.append(metadata["label"])
			imagings_test.append(metadata["imaging"])

	clear_output(wait=True)
 
	targets_test = np.concatenate(targets_test).flatten()
	labels_test = np.concatenate(labels_test).flatten()
	imagings_test = np.concatenate(imagings_test).flatten()

	test_df = pd.DataFrame.from_dict(
		{
			"signature": signatures_list,
			"target": data_loader.dataset.label_encoder.inverse_transform(targets_test),
			"imaging": imagings_test,
			"label": labels_test,
		}
	)
	return test_df, config

In [ ]:
run_ids = load_ids_from_registry()
test_data = [(load_test_df(run_id)) for run_id in run_ids]

save_signatures_dir = ensure_dir(configs.BASE_DIR / "saved/signatures")

for test_df, config in test_data:
	name = get_plot_name(config)
	plot_signatures(test_df["signature"].to_numpy(), test_df["target"].to_numpy(), title=name) 
	save_path = save_signatures_dir / f"{name}.png"
	plt.savefig(save_path)